In [5]:
# Upload CSVs
import pandas as pd

df1 = pd.read_csv ("../demo_database_creation/exported_views/investments.list_orders.csv", index_col=0, parse_dates=["date"])
df2 = pd.read_csv ("../demo_database_creation/exported_views/investments.table_for_alocations.csv", index_col=0)
df3 = pd.read_csv ("../demo_database_creation/exported_views/investments.table_for_geographies.csv", index_col=0)

df1

,order_id,buy_id,open_position,product_id,product_name,ticker,code_stockexc,date,hour,transaction_type,quantity,currency,price_unit,cambio_rate,comissions,taxes
0,1,1,False,1,Apple Inc,AAPL,NDQ,2022-05-18,09:05:00,BUY,20.00,USD,140.82,1.0519,3.0,28.16
1,2,2,True,1,Apple Inc,AAPL,NDQ,2022-06-02,10:13:00,BUY,10.00,USD,151.21,1.0706,3.0,15.12
2,3,1,False,1,Apple Inc,AAPL,NDQ,2023-11-27,15:04:00,SELL,20.00,USD,189.79,1.0737,3.0,97.94
3,4,3,True,2,Alphabet Inc,ABEC,TDG,2025-06-23,08:43:00,BUY,10.00,EUR,145.32,NaN,3.0,14.53
4,5,3,True,2,Alphabet Inc,ABEC,TDG,2025-09-15,07:34:00,Dividend,10.00,USD,0.21,1.1896,NaN,0.18
5,6,4,True,3,Vanguard FTSE All-World UCITS ETF USD Acc,VWCE,XET,2025-08-01,15:33:00,BUY,10.00,EUR,132.24,NaN,1.0,17.46
6,7,5,True,3,Vanguard FTSE All-World UCITS ETF USD Acc,VWCE,XET,2025-10-17,10:31:00,BUY,10.00,EUR,138.60,NaN,1.0,18.30
7,8,6,True,4,MercadoLibre Inc,MELI,NDQ,2025-11-25,15:49:00,BUY,1.00,USD,2006.06,1.1572,3.0,17.34
8,9,7,True,5,Deutsche Boerse Commodities Xetra-Gold ETC,4GLD,XET,2025-11-25,15:51:00,BUY,10.00,EUR,114.79,NaN,5.0,11.48
9,10,8,True,6,Bitcoin,BTC,none,2025-11-25,15:52:00,BUY,0.05,EUR,74723.56,NaN,1.0,NaN


In [40]:
import yfinance as yf

# These functions serve the purpose of obtaining the current stock prices, cambio rates, etc

def last_close_price (ticker):
    """
    Returns the last closing price in Yahoo Finance
    """
    x = yf.Ticker(ticker)
    return x.info ["regularMarketPreviousClose"]

cambio_eur_usd = last_close_price("EURUSD=X")
cambio_eur_dkk = last_close_price("EURDKK=X")

def last_close_df (dataframe):
    """
    Returns the last closing price in Yahoo Finance, but applied to a whole dataframe
    """
    dataframe.loc [dataframe["code_stockexc"] == "OMK", "ticker_stockexc"] = dataframe["ticker"] + ".CO"
    dataframe.loc [dataframe["code_stockexc"] == "EAM", "ticker_stockexc"] = dataframe["ticker"] + ".AS"
    dataframe.loc [dataframe["code_stockexc"] == "ELI", "ticker_stockexc"] = dataframe["ticker"] + ".LS"
    dataframe.loc [dataframe["code_stockexc"] == "MIL", "ticker_stockexc"] = dataframe["ticker"] + ".MI"
    dataframe.loc [dataframe["code_stockexc"] == "NDQ", "ticker_stockexc"] = dataframe["ticker"]
    dataframe.loc [dataframe["code_stockexc"] == "NSY", "ticker_stockexc"] = dataframe["ticker"]
    dataframe.loc [dataframe["code_stockexc"] == "TDG", "ticker_stockexc"] = dataframe["ticker"] + ".DE"
    dataframe.loc [dataframe["code_stockexc"] == "XET", "ticker_stockexc"] = dataframe["ticker"] + ".DE"
    dataframe.loc [dataframe["code_stockexc"] == "FWB", "ticker_stockexc"] = dataframe["ticker"] + ".F"
    
    dataframe.loc [dataframe["product_id"] == 6, "ticker_stockexc"] = "BTC-EUR"
    
    for i in range (len(dataframe)):
        dataframe.loc [i,"current_closing_price"] = (yf.Ticker(dataframe.loc [i,"ticker_stockexc"])).info ["regularMarketPreviousClose"]

    return dataframe

#### Generating a dataframe with all orders

In [17]:
import pandas as pd

# Import the view
df_orders = df1.copy()

# Fill the NULL values in the comissions, taxes, and cambio rates
df_orders ["cambio_rate"] = df_orders["cambio_rate"].fillna(1)
df_orders [["comissions", "taxes"]] = df_orders[["comissions", "taxes"]].fillna(0)

# Calculates the total spent in comissions and taxes
df_orders ["total_comissions_taxes"] = df_orders["comissions"] + df_orders["taxes"]

# Anxiliary columns that show the total net and gross value of each transaction
df_orders ["total_invested"] = 0.0
df_orders.loc [df_orders["transaction_type"] == "BUY", "total_invested"] = (df_orders ["quantity"] * df_orders ["price_unit"]) / df_orders ["cambio_rate"]
df_orders ["transaction_value_gross"] = 0.0
df_orders.loc [df_orders["transaction_type"] == "BUY", "transaction_value_gross"] = df_orders ["quantity"] * df_orders ["price_unit"] / df_orders ["cambio_rate"]
df_orders.loc [df_orders["transaction_type"] == "SELL", "transaction_value_gross"] = (-1) * (df_orders ["quantity"] * df_orders ["price_unit"] / df_orders ["cambio_rate"])
df_orders.loc [df_orders["transaction_type"] == "Dividend", "transaction_value_gross"] = (-1) * df_orders ["quantity"] * df_orders ["price_unit"] / df_orders ["cambio_rate"]
df_orders ["transaction_value_net"] = 0.0
df_orders.loc [df_orders["transaction_type"] == "BUY", "transaction_value_net"] = df_orders ["quantity"] * df_orders ["price_unit"] / df_orders ["cambio_rate"] - df_orders ["total_comissions_taxes"]
df_orders.loc [df_orders["transaction_type"] == "SELL", "transaction_value_net"] = (-1) * (df_orders ["quantity"] * df_orders ["price_unit"] / df_orders ["cambio_rate"]) - df_orders ["total_comissions_taxes"]
df_orders.loc [df_orders["transaction_type"] == "Dividend", "transaction_value_net"] = (-1) * df_orders ["quantity"] * df_orders ["price_unit"] / df_orders ["cambio_rate"] - df_orders ["total_comissions_taxes"]

# Coluna status
df_orders.loc [df_orders["open_position"] == False, "status"] = "Closed"
df_orders.loc [df_orders["open_position"] == True, "status"] = "Open"

df_orders.to_csv("exported_dataframes/df_orders.csv", encoding='utf-8-sig')

df_orders

,order_id,buy_id,open_position,product_id,product_name,ticker,code_stockexc,date,hour,transaction_type,...,currency,price_unit,cambio_rate,comissions,taxes,total_comissions_taxes,total_invested,transaction_value_gross,transaction_value_net,status
0,1,1,False,1,Apple Inc,AAPL,NDQ,2022-05-18,09:05:00,BUY,...,USD,140.82,1.0519,3.0,28.16,31.16,2677.440821,2677.440821,2646.280821,Closed
1,2,2,True,1,Apple Inc,AAPL,NDQ,2022-06-02,10:13:00,BUY,...,USD,151.21,1.0706,3.0,15.12,18.12,1412.385578,1412.385578,1394.265578,Open
2,3,1,False,1,Apple Inc,AAPL,NDQ,2023-11-27,15:04:00,SELL,...,USD,189.79,1.0737,3.0,97.94,100.94,0.000000,-3535.251933,-3636.191933,Closed
3,4,3,True,2,Alphabet Inc,ABEC,TDG,2025-06-23,08:43:00,BUY,...,EUR,145.32,1.0000,3.0,14.53,17.53,1453.200000,1453.200000,1435.670000,Open
4,5,3,True,2,Alphabet Inc,ABEC,TDG,2025-09-15,07:34:00,Dividend,...,USD,0.21,1.1896,0.0,0.18,0.18,0.000000,-1.765299,-1.945299,Open
5,6,4,True,3,Vanguard FTSE All-World UCITS ETF USD Acc,VWCE,XET,2025-08-01,15:33:00,BUY,...,EUR,132.24,1.0000,1.0,17.46,18.46,1322.400000,1322.400000,1303.940000,Open
6,7,5,True,3,Vanguard FTSE All-World UCITS ETF USD Acc,VWCE,XET,2025-10-17,10:31:00,BUY,...,EUR,138.60,1.0000,1.0,18.30,19.30,1386.000000,1386.000000,1366.700000,Open
7,8,6,True,4,MercadoLibre Inc,MELI,NDQ,2025-11-25,15:49:00,BUY,...,USD,2006.06,1.1572,3.0,17.34,20.34,1733.546492,1733.546492,1713.206492,Open
8,9,7,True,5,Deutsche Boerse Commodities Xetra-Gold ETC,4GLD,XET,2025-11-25,15:51:00,BUY,...,EUR,114.79,1.0000,5.0,11.48,16.48,1147.900000,1147.900000,1131.420000,Open
9,10,8,True,6,Bitcoin,BTC,none,2025-11-25,15:52:00,BUY,...,EUR,74723.56,1.0000,1.0,0.00,1.00,3736.178000,3736.178000,3735.178000,Open


#### Generating a dataframe with the profit and yield per product

In [ ]:
df_prod_profit = pd.read_csv ("exported_dataframes/df_orders.csv", index_col="order_id", parse_dates=["date"])
df_prod_profit = df_prod_profit.reset_index()

# Filtering the columns that manner, and then the open positions only
df_prod_profit = df_prod_profit [ ["open_position", "product_id", "product_name", "total_invested", "transaction_value_gross", "transaction_value_net", "total_comissions_taxes"] ]
df_prod_profit = df_prod_profit [df_prod_profit ["open_position"] == False]

# Treatments
df_prod_profit ["total_invested_with_taxas"] = df_prod_profit ["total_invested"] + df_prod_profit ["total_comissions_taxes"]
df_prod_profit ["transaction_value_gross"] = df_prod_profit ["transaction_value_gross"] * (-1)
df_prod_profit ["transaction_value_net"] = df_prod_profit ["transaction_value_gross"] - df_prod_profit ["total_comissions_taxes"]

# Grouping by product
df_prod_profit = df_prod_profit.groupby (["product_id", "product_name"]) [["total_invested", "total_invested_with_taxas", "transaction_value_gross", "transaction_value_net"]].sum()
# Calculate yield
df_prod_profit ["yield"] = df_prod_profit ["transaction_value_net"] / df_prod_profit ["total_invested_with_taxas"]
# Round values
df_prod_profit[["total_invested", "total_invested_with_taxas", "transaction_value_gross", "transaction_value_net", "yield"]] = df_prod_profit[["total_invested", "total_invested_with_taxas", "transaction_value_gross", "transaction_value_net", "yield"]].round(3)

df_prod_profit.to_csv("exported_dataframes/df_prod_profit.csv", encoding='utf-8-sig')

df_prod_profit

,,total_invested,total_invested_with_taxas,transaction_value_gross,transaction_value_net,yield
product_id,product_name,,,,,
1,Apple Inc,2677.441,2809.541,857.811,725.711,0.258


#### Generating a dataframe with all open positions

In [38]:
df_open_positions = pd.read_csv ("exported_dataframes/df_orders.csv", index_col="order_id", parse_dates=["date"])
df_open_positions = df_open_positions.reset_index()

# Filtering the columns that manner, and then the open positions only
df_open_positions = df_open_positions [ ["buy_id", "open_position", "product_id", "product_name", "ticker", "code_stockexc", "transaction_type", "currency", "quantity", "price_unit"] ]
df_open_positions = df_open_positions [df_open_positions ["open_position"] == True]

# Treatments before grouping: 1) remove the rows related with dividends (mess up the next calculation); 2) make the "quantity" of the "SELLs" negative, and its unit_price = 0
df_open_positions = df_open_positions [df_open_positions ["transaction_type"] != "Dividend"]
df_open_positions.loc [df_open_positions["transaction_type"] == "SELL", "quantity"] = df_open_positions ["quantity"] * (-1)
df_open_positions.loc [df_open_positions["transaction_type"] == "SELL", "price_unit"] = 0

# Grouping by buy_id. Note that the unit price can be used in this sum because I classified it as 0 when we have a SELL
df_open_positions = df_open_positions.groupby (["buy_id", "product_id", "product_name", "ticker", "code_stockexc", "currency"]) [["quantity", "price_unit"]].sum()
df_open_positions = df_open_positions.reset_index()

# Determine equilibrium prices by product
df_open_positions ["invested_value"] = df_open_positions ["quantity"] * df_open_positions ["price_unit"]
df_open_positions = df_open_positions.groupby (["product_id", "product_name", "ticker", "code_stockexc", "currency"]) [["quantity", "invested_value"]].sum()
df_open_positions ["equilibrium_price"] = df_open_positions ["invested_value"] / df_open_positions ["quantity"]
df_open_positions = df_open_positions.reset_index()

# Obtain current prices
df_open_positions = last_close_df (df_open_positions)
df_open_positions ["current_value"] = df_open_positions ["quantity"] * df_open_positions ["current_closing_price"]
df_open_positions ["current_gross_profit"] = df_open_positions ["current_value"] - df_open_positions ["invested_value"]

# Round floats
df_open_positions[["quantity", "equilibrium_price", "current_closing_price", "invested_value", "current_value", "current_gross_profit"]] = df_open_positions[["quantity", "equilibrium_price", "current_closing_price", "invested_value", "current_value", "current_gross_profit"]].round(3)

# Filtering the columns that manner
df_open_positions = df_open_positions [ ["product_name", "quantity", "equilibrium_price", "current_closing_price", "currency", "invested_value", "current_value", "current_gross_profit"] ]
df_open_positions = df_open_positions.set_index("product_name")

df_open_positions.to_csv("exported_dataframes/df_open_positions.csv", encoding='utf-8-sig')

df_open_positions

,quantity,equilibrium_price,current_closing_price,currency,invested_value,current_value,current_gross_profit
product_name,,,,,,,
Apple Inc,10.00,151.21,275.920,USD,1512.100,2759.200,1247.100
Alphabet Inc,10.00,145.32,272.250,EUR,1453.200,2722.500,1269.300
Vanguard FTSE All-World UCITS ETF USD Acc,20.00,135.42,142.460,EUR,2708.400,2849.200,140.800
MercadoLibre Inc,1.00,2006.06,2008.920,USD,2006.060,2008.920,2.860
Deutsche Boerse Commodities Xetra-Gold ETC,10.00,114.79,114.340,EUR,1147.900,1143.400,-4.500
Bitcoin,0.05,74723.56,76619.660,EUR,3736.178,3830.983,94.805
Goldman Sachs Access China Government Bond UCITS ETF USD (Dist),10.00,47.19,47.105,EUR,471.900,471.050,-0.850


#### Generating a dataframe with the weight of each product in the wallet

In [ ]:
df_weights = pd.read_csv ("exported_dataframes/df_open_positions.csv", index_col=0)
df_weights = df_weights.reset_index()

# Transform the current values in euros
df_weights.loc [df_weights["currency"] == "USD", "current_value"] = df_weights ["current_value"] / cambio_eur_usd
df_weights.loc [df_weights["currency"] == "DKK", "current_value"] = df_weights ["current_value"] / cambio_eur_dkk

# Calculate the weights of each product in the wallet
soma_todos_invs = df_weights["current_value"].sum()
df_weights ["weight_in_wallet"] = df_weights["current_value"] / soma_todos_invs

# Change names of some products (relevant for visualization)
df_weights.loc [df_weights["product_name"] == "Vanguard FTSE All-World UCITS ETF USD Acc", "product_name"] = "Vanguard All-World ETF"
df_weights.loc [df_weights["product_name"] == "Deutsche Boerse Commodities Xetra-Gold ETC", "product_name"] = "Xetra Gold ETC"
df_weights.loc [df_weights["product_name"] == "Goldman Sachs Access China Government Bond UCITS ETF USD (Dist)", "product_name"] = "Goldman Sachs China Gov Bonds"

# Filter columns that matter
df_weights = df_weights [ ["product_name", "current_value", "weight_in_wallet"] ]
df_weights = df_weights.set_index("product_name")
# Round floats
df_weights[["current_value", "weight_in_wallet"]] = df_weights[["current_value", "weight_in_wallet"]].round(3)

df_weights.to_csv("exported_dataframes/df_weights.csv", encoding='utf-8-sig')

df_weights

,current_value,weight_in_wallet
product_name,,
Apple Inc,2393.330,0.158
Alphabet Inc,2722.500,0.180
Vanguard All-World ETF,2849.200,0.188
MercadoLibre Inc,1742.537,0.115
Xetra Gold ETC,1143.400,0.075
Bitcoin,3830.983,0.253
Goldman Sachs China Gov Bonds,471.050,0.031


#### Generating a dataframe with the geographic distribution of investments

In [50]:
# Import view
df_geographies = df3.copy()

# Filter relevant columns
df_geographies = df_geographies [ ["order_id", "buy_id", "open_position", "product_id", "product_name", "ticker", "code_stockexc", "transaction_type", "currency", "quantity", "price_unit", "geography_id", "perc_north_america", "perc_latin_america", "perc_europe", "perc_mena", "perc_east_se_asia", "perc_oceania", "perc_other_geo"] ]
# Filter the open positions only
df_geographies = df_geographies [df_geographies ["open_position"] == True]

# Treatments before grouping: 1) remove the rows related with dividends (messes up the next calculation); 2) make the "quantity" of the "SELLs" negative, and its unit_price = 0
df_geographies = df_geographies [df_geographies ["transaction_type"] != "Dividend"]
df_geographies.loc [df_geographies["transaction_type"] == "SELL", "quantity"] = df_geographies ["quantity"] * (-1)
df_geographies.loc [df_geographies["transaction_type"] == "SELL", "price_unit"] = 0

# Grouping by order. Reminder: the unit price can be used in this sum because I classified it as 0 when we have a SELL
df_geographies = df_geographies.groupby (["buy_id", "product_id", "product_name", "ticker", "code_stockexc", "currency", "geography_id", "perc_north_america", "perc_latin_america", "perc_europe", "perc_mena", "perc_east_se_asia", "perc_oceania", "perc_other_geo"]) [["quantity", "price_unit"]].sum()
df_geographies = df_geographies.reset_index()

# Determine the total invested value by product, then do the respective grouping
df_geographies ["invested_value"] = df_geographies ["quantity"] * df_geographies ["price_unit"]
df_geographies = df_geographies.groupby (["product_id", "product_name", "ticker", "code_stockexc", "currency", "geography_id", "perc_north_america", "perc_latin_america", "perc_europe", "perc_mena", "perc_east_se_asia", "perc_oceania", "perc_other_geo"]) [["quantity", "price_unit"]].sum()
df_geographies = df_geographies.reset_index()

# Add current prices
df_geographies = last_close_df (df_geographies)
df_geographies ["current_value"] = df_geographies ["quantity"] * df_geographies ["current_closing_price"]

# Transform the current values in euros
df_geographies.loc [df_geographies["currency"] == "USD", "current_value"] = df_geographies ["current_value"] / cambio_eur_usd
df_geographies.loc [df_geographies["currency"] == "DKK", "current_value"] = df_geographies ["current_value"] / cambio_eur_dkk

# Calculate the weight of each geography in the wallet
total_invest = df_geographies["current_value"].sum()
df_geographies ["weight_in_wallet"] = df_geographies["current_value"] / total_invest

# Filter relevant columns
df_geographies = df_geographies [["perc_north_america", "perc_latin_america", "perc_europe", "perc_mena", "perc_east_se_asia", "perc_oceania", "perc_other_geo", "weight_in_wallet"]]
df_geographies = df_geographies.set_index("weight_in_wallet")

# Treatment of the data - multiply the weight by each field in its row
df_geographies = df_geographies.mul(df_geographies.index, axis=0).mul(0.01, axis=0)
df_geographies = df_geographies.reset_index()

# Grouping of data
df_geographies = df_geographies [["perc_north_america", "perc_latin_america", "perc_europe", "perc_mena", "perc_east_se_asia", "perc_oceania", "perc_other_geo"]].sum()
df_geographies = df_geographies.reset_index()
df_geographies = df_geographies.rename (columns = {"index": "geo", 0: "weight"})

# Renaming of geographies
df_geographies ["geo"] = ["North America", "Latin America", "Europa", "MENA", "East and SE Asia", "Oceania", "Other"]

df_geographies.to_csv("exported_dataframes/df_geographies.csv", encoding='utf-8-sig')

df_geographies

,geo,weight
0,North America,0.461711
1,Latin America,0.116876
2,Europa,0.028204
3,MENA,0.001880
4,East and SE Asia,0.057410
5,Oceania,0.003761
6,Other,0.001880


#### Generating a dataframe with the alocation by type of asset

In [55]:
# Import view
df_aloc_type = df2.copy()

# Filter relevant columns
df_aloc_type = df_aloc_type [ ["open_position", "product_id", "product_name", "ticker", "code_stockexc", "transaction_type", "currency", "quantity", "price_unit", "alocation_id", "perc_stocks", "perc_bonds", "perc_gold", "perc_crypto", "perc_other_fin_instruments"] ]
# Filter the open positions only
df_aloc_type = df_aloc_type [df_aloc_type ["open_position"] == True]

# Treatments before grouping: 1) remove the rows related with dividends (mess up the next calculation); 2) make the "quantity" of the "SELLs" negative, and its unit_price = 0
df_aloc_type = df_aloc_type [df_aloc_type ["transaction_type"] != "Dividend"]
df_aloc_type.loc [df_aloc_type["transaction_type"] == "SELL", "quantity"] = df_aloc_type ["quantity"] * (-1)
df_aloc_type.loc [df_aloc_type["transaction_type"] == "SELL", "price_unit"] = 0

# Grouping by order. Reminder: the unit price can be used in this sum because I classified it as 0 when we have a SELL
df_aloc_type = df_aloc_type.groupby (["product_id", "product_name", "ticker", "code_stockexc", "currency", "alocation_id", "perc_stocks", "perc_bonds", "perc_gold", "perc_crypto", "perc_other_fin_instruments"]) [["quantity", "price_unit"]].sum()
df_aloc_type = df_aloc_type.reset_index()

# Determine the total invested value by product, then do the respective grouping
df_aloc_type ["invested_value"] = df_aloc_type ["quantity"] * df_aloc_type ["price_unit"]
df_aloc_type = df_aloc_type.groupby (["product_id", "product_name", "ticker", "code_stockexc", "currency", "alocation_id", "perc_stocks", "perc_bonds", "perc_gold", "perc_crypto", "perc_other_fin_instruments"]) [["quantity", "invested_value"]].sum()
df_aloc_type = df_aloc_type.reset_index()

# Add current prices
df_aloc_type = last_close_df (df_aloc_type)
df_aloc_type ["current_value"] = df_aloc_type ["quantity"] * df_aloc_type ["current_closing_price"]

# Transform the current values in euros
df_aloc_type.loc [df_aloc_type["currency"] == "USD", "current_value"] = df_aloc_type ["current_value"] / cambio_eur_usd
df_aloc_type.loc [df_aloc_type["currency"] == "DKK", "current_value"] = df_aloc_type ["current_value"] / cambio_eur_dkk

# Calculate the weight of each asset type in the wallet
total_invest_al = df_aloc_type["current_value"].sum()
df_aloc_type ["weight"] = df_aloc_type["current_value"] / total_invest_al

# Filter relevant columns
df_aloc_type = df_aloc_type [["perc_stocks", "perc_bonds", "perc_gold", "perc_crypto", "perc_other_fin_instruments", "weight"]]
df_aloc_type = df_aloc_type.set_index("weight")

# Treatment of the data - multiply the weight by each field in its row
df_aloc_type = df_aloc_type.mul(df_aloc_type.index, axis=0).mul(0.01, axis=0)
df_aloc_type = df_aloc_type.reset_index()

# Grouping of data
df_aloc_type = df_aloc_type [["perc_stocks", "perc_bonds", "perc_gold", "perc_crypto", "perc_other_fin_instruments"]].sum()
df_aloc_type = df_aloc_type.reset_index()
df_aloc_type = df_aloc_type.rename (columns = {"index": "aloc_type", 0: "weight"})

df_aloc_type ["aloc_type"] = ['Stocks', 'Bonds', 'Gold', 'Crypto', 'Other']

df_aloc_type.to_csv("exported_dataframes/df_aloc_type.csv", encoding='utf-8-sig')

df_aloc_type

,aloc_type,weight
0,Stocks,0.508452
1,Bonds,0.155561
2,Gold,0.075457
3,Crypto,0.252820
4,Other,0.007709
